<a href="https://colab.research.google.com/github/MouadFiali/Health-Recommendation-System/blob/main/Notebooks/collaborative_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center>Patient - Disease Recommendation system</center>

In [1]:
# import necessary libraries
import pandas as pd
import dask.dataframe as dd
import numpy as np
import ast
from sklearn.preprocessing import MultiLabelBinarizer

## Loading dataset using Dask

In [2]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [ ]:
# load the dataset
# json files
release_evidences_df = pd.read_json('/content/gdrive/MyDrive/Colab/release_evidences.json')
release_conditions_df = pd.read_json('/content/gdrive/MyDrive/Colab/release_conditions.json')
# csv files (the files we need to train the models)
training_df = dd.read_csv('/content/gdrive/MyDrive/Colab/release_train_patients.csv')
testing_df = dd.read_csv('/content/gdrive/MyDrive/Colab/release_test_patients.csv')
validate_df = dd.read_csv('/content/gdrive/MyDrive/Colab/release_validate_patients.csv')

In [ ]:
# save the dataset (training, testing and validation) into one file
dataset_df = dd.concat([training_df, testing_df, validate_df])
dataset_df.to_csv('/content/gdrive/MyDrive/Colab/dataset.csv', single_file=True, index=False)

['/content/gdrive/MyDrive/Colab/dataset.csv']

## Exploring data

In [ ]:
# Let's see the shape of the dataframes
print('release_evidences_df shape: ', release_evidences_df.shape)
print('release_conditions_df shape: ', release_conditions_df.shape)
print(f'training_df shape: ({training_df.shape[0].compute()}, {training_df.shape[1]})')
print(f'testing_df shape: ({testing_df.shape[0].compute()}, {testing_df.shape[1]})')
print(f'validate_df shape: ({validate_df.shape[0].compute()}, {validate_df.shape[1]})')

release_evidences_df shape:  (9, 223)
release_conditions_df shape:  (7, 49)
training_df shape: (1025602, 6)
testing_df shape: (134529, 6)
validate_df shape: (132448, 6)


In [ ]:
training_df.head()

,AGE,DIFFERENTIAL_DIAGNOSIS,SEX,PATHOLOGY,EVIDENCES,INITIAL_EVIDENCE
0,18,"[['Bronchite', 0.19171203430383882], ['Pneumon...",M,IVRS ou virémie,"['crowd', 'diaph', 'douleurxx', 'douleurxx_car...",fievre
1,21,"[['VIH (Primo-infection)', 0.5189500564407601]...",M,VIH (Primo-infection),"['adp_dlr', 'atcd_its', 'diaph', 'diarrhee', '...",diaph
2,19,"[['Bronchite', 0.11278064619119596], ['Pneumon...",F,Pneumonie,"['douleurxx', 'douleurxx_carac_@_un_coup_de_co...",expecto
3,34,"[['IVRS ou virémie', 0.23859396799565236], ['C...",F,IVRS ou virémie,"['crowd', 'douleurxx', 'douleurxx_carac_@_une_...",douleurxx
4,36,"[['IVRS ou virémie', 0.23677812769175735], ['P...",M,IVRS ou virémie,"['dayc', 'diaph', 'douleurxx', 'douleurxx_cara...",toux


In [ ]:
testing_df.head()

,AGE,DIFFERENTIAL_DIAGNOSIS,SEX,PATHOLOGY,EVIDENCES,INITIAL_EVIDENCE
0,49,"[['Bronchite', 0.20230062181160519], ['RGO', 0...",F,RGO,"['douleurxx', 'douleurxx_carac_@_lancinante_/_...",toux
1,2,"[['Asthme exacerbé ou bronchospasme', 0.080220...",M,Bronchite,"['douleurxx', 'douleurxx_carac_@_une_brûlure_o...",douleurxx
2,49,"[['Réaction dystonique aïgue', 0.6267050848165...",M,Réaction dystonique aïgue,"['antipsy_récent', 'laryngospasme', 'nau_psy_r...",laryngospasme
3,64,"[['Bronchite', 0.2748608320637265], ['Laryngit...",M,Laryngite aigue,"['crowd', 'dayc', 'douleurxx', 'douleurxx_cara...",douleurxx
4,70,"[['IVRS ou virémie', 0.21257615919851483], ['P...",F,IVRS ou virémie,"['contact', 'diaph', 'douleurxx', 'douleurxx_c...",toux


In [ ]:
validate_df.head()

,AGE,DIFFERENTIAL_DIAGNOSIS,SEX,PATHOLOGY,EVIDENCES,INITIAL_EVIDENCE
0,55,"[['Anémie', 0.25071110167158567], ['Fibrillati...",F,Anémie,"['Mauv_aliment', 'atcd_anem', 'atcd_fam_anem',...",pale
1,10,"[['Syndrome de Guillain-Barré', 0.135558991316...",F,Attaque de panique,"['anxiete_s', 'atcdpsyfam', 'diaph', 'douleurx...",psy_depers
2,68,[['Possible influenza ou syndrome virémique ty...,F,Possible influenza ou syndrome virémique typique,"['diaph', 'douleurxx', 'douleurxx_carac_@_une_...",douleurxx
3,13,"[['Anémie', 0.18697604010451876], ['Fibrillati...",M,Anémie,"['Mauv_aliment', 'atcd_anem', 'atcd_fam_anem',...",douleurxx
4,48,"[['Syndrome de Boerhaave', 1.0]]",M,Syndrome de Boerhaave,"['douleurxx', 'douleurxx_carac_@_déchirante', ...",douleurxx


In [ ]:
# Concat all the dfs into one df to make the processing easier
df = dd.concat([validate_df, training_df, testing_df])

In [ ]:
df.to_csv('/content/gdrive/MyDrive/Colab/dataset.csv', single_file=True, index=False)

## Pre-processing

In [ ]:
dataset_df = pd.read_csv('/content/gdrive/MyDrive/Colab/dataset.csv', chunksize=129257) # data size / 10

In [ ]:
# first let's get all the possible evidences
# read the csv using dask for faster performance
temp_df = dd.read_csv('/content/gdrive/MyDrive/Colab/dataset.csv')

# get the evidences
all_evidences = temp_df['EVIDENCES'].apply(ast.literal_eval, meta=('object'))

unique_values = set()
for sublist in all_evidences:
    unique_values.update(sublist)

temp_df = None

/usr/local/lib/python3.10/dist-packages/dask/dataframe/core.py:6751: FutureWarning: Meta is not valid, `map_partitions` and `map_overlap` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.
  warnings.warn(


In [ ]:
# Create a MultiLabelBinarizer object
mlb = MultiLabelBinarizer(classes=list(unique_values))

In [ ]:
chunk_number = 1

for chunk in dataset_df:
    print('Processing chunk', chunk_number, '----------------------------------')
    
    # Create a MultiLabelBinarizer object
    mlb = MultiLabelBinarizer(classes=list(unique_values))

    # Convert the evidence column into a list of lists
    evidence_lists = chunk['EVIDENCES'].apply(ast.literal_eval)

    # Apply one-hot encoding to the evidence lists
    print('Creating encoded evidences dataframe using one-hot...')
    evidence_encoded = pd.DataFrame(mlb.fit_transform(evidence_lists), columns=mlb.classes_)
    print('Encoded evidences dataframe created.')

    print('size of chunk: ', chunk.shape)
    print('size of encoded: ', evidence_encoded.shape)

    # drop indexes to avoid null values
    chunk.reset_index(drop=True, inplace=True)
    evidence_encoded.reset_index(drop=True, inplace=True)

    # Concatenate the encoded evidence columns with the chunk
    processed_chunk = pd.concat([chunk, evidence_encoded], axis=1)
    print('Chunk processed as pandas dataframe')

    processed_chunk.to_csv('/content/gdrive/MyDrive/Colab/Dataset chunks/chunk_'+ str(chunk_number) + '.csv', index=False)
    print('Chunk added to csv.')
    chunk_number+=1

Processing chunk 1 ----------------------------------
Creating encoded evidences dataframe using one-hot...
Encoded evidences dataframe created.
size of chunk:  (129257, 6)
size of encoded:  (129257, 516)
Chunk processed as pandas dataframe
Chunk added to csv.
Processing chunk 2 ----------------------------------
Creating encoded evidences dataframe using one-hot...
Encoded evidences dataframe created.
size of chunk:  (129257, 6)
size of encoded:  (129257, 516)
Chunk processed as pandas dataframe
Chunk added to csv.
Processing chunk 3 ----------------------------------
Creating encoded evidences dataframe using one-hot...
Encoded evidences dataframe created.
size of chunk:  (129257, 6)
size of encoded:  (129257, 516)
Chunk processed as pandas dataframe
Chunk added to csv.
Processing chunk 4 ----------------------------------
Creating encoded evidences dataframe using one-hot...
Encoded evidences dataframe created.
size of chunk:  (129257, 6)
size of encoded:  (129257, 516)
Chunk proces

In [47]:
ddf = dd.read_csv('/content/gdrive/MyDrive/Colab/Dataset chunks/*.csv')

In [ ]:
ddf.isnull().values.any().compute()

False

In [ ]:
ddf.shape[0].compute(), ddf.shape[1]

(1292579, 522)

In [ ]:
ddf.head()

,AGE,DIFFERENTIAL_DIAGNOSIS,SEX,PATHOLOGY,EVIDENCES,INITIAL_EVIDENCE,z80.0,trav1_@_AmerN,protu_langue,lesions_peau_elevee_@_7,...,lesions_peau_prurit_@_2,oedeme_endroitducorps_@_coté_lateral_du_pied_G_,douleurxx_endroitducorps_@_aisselle_D_,ains,douleurxx_endroitducorps_@_amygdale_D_,cont_coq,oedeme,boire_ped,douleurxx_endroitducorps_@_face_dorsale_du_poignet_D_,oedeme_endroitducorps_@_plante_du_pied_G_
0,18,"[['Bronchite', 0.19171203430383882], ['Pneumon...",M,IVRS ou virémie,"['crowd', 'diaph', 'douleurxx', 'douleurxx_car...",fievre,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,21,"[['VIH (Primo-infection)', 0.5189500564407601]...",M,VIH (Primo-infection),"['adp_dlr', 'atcd_its', 'diaph', 'diarrhee', '...",diaph,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,19,"[['Bronchite', 0.11278064619119596], ['Pneumon...",F,Pneumonie,"['douleurxx', 'douleurxx_carac_@_un_coup_de_co...",expecto,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,34,"[['IVRS ou virémie', 0.23859396799565236], ['C...",F,IVRS ou virémie,"['crowd', 'douleurxx', 'douleurxx_carac_@_une_...",douleurxx,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,36,"[['IVRS ou virémie', 0.23677812769175735], ['P...",M,IVRS ou virémie,"['dayc', 'diaph', 'douleurxx', 'douleurxx_cara...",toux,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
ddf['diaph'].compute().iloc[4]

1

In [ ]:
# check the shape of the combined data
print(f'Total number of patients: {training_df.shape[0].compute() + testing_df.shape[0].compute() + validate_df.shape[0].compute()}')
print(f'combined df shape: ({ddf.shape[0].compute()}, {ddf.shape[1]})')

Total number of patients: 1292579
combined df shape: (1292579, 6)


In [4]:
# get the data size
data_size = ddf.shape[0].compute()

In [ ]:
ddf.repartition(partition_size="100MB")

,AGE,DIFFERENTIAL_DIAGNOSIS,SEX,PATHOLOGY,EVIDENCES,INITIAL_EVIDENCE,z80.0,trav1_@_AmerN,protu_langue,lesions_peau_elevee_@_7,z77.22,lesions_peau_elevee_@_6,perinatality,f17.210,douleurxx_endroitducorps_@_orteil__3__G_,j33,crach_sg,douleurxx_endroitducorps_@_hanche_G_,pertes_vag,douleurxx_irrad_@_arrière_de_tête,trav1_@_AsieSSE,douleurxx_endroitducorps_@_pouce_D_,lesions_peau_endroitducorps_@_vagin,rds_anorexie,douleurxx_endroitducorps_@_petit_orteil__4__G_,psy_depers,douleurxx_endroitducorps_@_oreille_G_,B34.9,douleurxx_endroitducorps_@_côté_du_thorax_G_,histfammigraine,douleurxx_endroitducorps_@_sein_G_,douleurxx_endroitducorps_@_sous_la_machoire,melena,vaccination,dysarthrie,douleurxx_endroitducorps_@_pubis,sialorhee,douleurxx_soudain_@_0,douleurxx_endroitducorps_@_côté_du_cou_G_,I30,douleurxx_endroitducorps_@_creux_poplité_D_,douleurxx_endroitducorps_@_plante_du_pied_D_,rural,lesions_peau_endroitducorps_@_bouche,scombroide,rx_vasodil,tmine,faiblesse_msmi,lesions_peau_endroitducorps_@_scrotum,lesions_peau_plusqu1cm_@_N,contact,douleurxx_endroitducorps_@_tibia_G_,wheez,lesions_peau_couleur_@_jaune,z84.89,allait_prol,perte_poids,douleurxx_intens_@_4,drogues_stimul,douleurxx_carac_@_sensible,douleurxx_endroitducorps_@_biceps_G_,lesions_peau_intens_@_5,move,i25.1,diplopie,lesions_peau_endroitducorps_@_cartilage_thyroidien,oedeme_endroitducorps_@_joue_G_,dayc,J81,regard_dévié,lesions_peau_intens_@_6,dyspn,douleurxx_carac_@_pénible,douleurxx_endroitducorps_@_doigt_majeur__G_,posttus_emesis,atcd_fam_anem,lesions_peau_prurit_@_4,douleurxx_endroitducorps_@_mollet_D_,f41.9,fievre,douleurxx_carac_@_une_crampe,j34.2,lesions_peau_endroitducorps_@_biceps_G_,larmes,ap_fk,douleurxx_carac_@_une_brûlure_ou_chaleur,expecto,dysphagie,ptose,douleurxx_endroitducorps_@_triceps_D_,lesions_peau_endroitducorps_@_ventre,faiblesse faciale,oedeme_endroitducorps_@_mollet_D_,douleurxx_precis_@_0,lesions_peau_prurit_@_10,i50,rectorragie,oedeme_endroitducorps_@_face_dorsale_du_pied_D_,douleurxx_irrad_@_nulle_part,volume_parole,oedeme_endroitducorps_@_tibia_G_,fatig_mod,lesions_peau_endroitducorps_@_fosse_iliaque_D_,douleurxx_endroitducorps_@_épaule_G_,nau_psy_recent,palpit,douleurxx_irrad_@_front,douleurxx_endroitducorps_@_cuisse_G_,douleurxx_endroitducorps_@_doigt_auriculaire__G_,douleurxx_intens_@_2,cortico,douleurxx_endroitducorps_@_cheville_G_,z82.49,lesions_peau_endroitducorps_@_thorax_postérieur_G_,trav1_@_AfriqN,prurit_occ,norvasc,douleurxx_endroitducorps_@_trachée,rhino_clair,apnee,oedeme_endroitducorps_@_coté_lateral_du_pied_D_,douleurxx_soudain_@_7,douleurxx_precis_@_10,drogues_IV,trav1_@_AfriqO,antipsy_récent,douleurxx_intens_@_8,k21,douleurxx_endroitducorps_@_joue_G_,atcdfam_mg,douleurxx_endroitducorps_@_orteil__2__G_,spasmes_msk,k74,douleurxx_endroitducorps_@_face_dorsale_du_pied_G_,vo_sg,douleurxx_endroitducorps_@_doigt_annulaire__D_,douleurxx_endroitducorps_@_omoplate_G_,toux_sev,pdc,drink_energie,douleurxx_carac_@_un_coup_de_couteau,lesions_peau_endroitducorps_@_colonne_dorsale,lesions_peau_endroitducorps_@_cheville_D_,lesions_peau_intens_@_3,laryngospasme,douleurxx_endroitducorps_@_épaule_D_,trav1_@_AfriqSS,sex_vih,douleurxx_endroitducorps_@_orteil__1__G_,tconst,douleurxx_intens_@_0,lesions_peau_endroitducorps_@_colonne_cervicale,lesions_peau_plusqu1cm_@_O,douleurxx_soudain_@_5,oedeme_endroitducorps_@_nez,lesions_peau_endroitducorps_@_fesse_D_,allergie_sev,lesions_peau_endroitducorps_@_cuisse_G_,douleurxx_endroitducorps_@_dessus_de_tête,dyspn_noct,lesions_peau_couleur_@_NA,lesions_peau_endroitducorps_@_cuisse_D_,oedeme_endroitducorps_@_cheville_G_,douleurxx_precis_@_2,lesions_peau_prurit_@_5,lesions_peau_couleur_@_rouge,douleurxx_endroitducorps_@_biceps_D_,douleurxx_carac_@_un_tiraillement,ap_valve,douleurxx_precis_@_9,douleurxx_irrad_@_épaule_G_,douleurxx,e10_e11,douleurxx_endroitducorps_@_omoplate_D_,douleurxx_endroitducorps_@_ischio_G_,itss_risque,lesions_peau_endroitducorps_@_gencive_supérieure,lesions_peau_endroitducorps_@_joue_intern

In [5]:
print('ddf npartitions: ', ddf.npartitions)
print('ddf divisions: ', ddf.divisions)

ddf npartitions:  31
ddf divisions:  (None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None)


In [ ]:
# Save the list of the evidences in case we need them
with open('/content/gdrive/MyDrive/Colab/evidences.txt', 'a', encoding='utf-8') as f:
  f.write(str(list(unique_values)))

In [ ]:
# check if the evidences are correctly distributed (naive check)
print('Patient 0 - Evidence (adp_dlr) : ')
print(ddf['adp_dlr'].compute().iloc[0])
print('Patient 1 - Evidence (adp_dlr) : ')
print(ddf['adp_dlr'].compute().iloc[1])
print('Patient 0 - Evidence (fievre) : ')
print(ddf['fievre'].compute().iloc[0])
print('Patient 1 - Evidence (fievre) : ')
print(ddf['fievre'].compute().iloc[1])

Patient 0 - Evidence (adp_dlr) : 
0
Patient 1 - Evidence (adp_dlr) : 
1
Patient 0 - Evidence (fievre) : 
1
Patient 1 - Evidence (fievre) : 
1


We can say that the evidences are correctly distributed using these examples. In fact we are sure that the evidence 'fievre' exists for patient 0 as it's an INITIAL_EVIDENCE. Same thing for 'adp_dlr' and the patient 1.
We can naively say that things went well and we can count on our binarizer.

In [ ]:
# Let's make another verif
ddf['INITIAL_EVIDENCE'].compute().iloc[67293] # Let's check an evidence that we are sure to exist

'rds_deg'

In [ ]:
# Let's check now for the value of this column
ddf['rds_deg'].compute().iloc[67293]

1

In [51]:
# replace M, F values by 0 and 1
ddf = ddf.map_partitions(pd.DataFrame.replace, {'M': 1, 'F': 0})

In [48]:
# We won't need the INITIAL_EVIDENCE & DIFFERENTIAL_DIAGNOSIS
# The 1st is just a redundant information (we already have it in the other columns)
# The 2nd is just a probability of each pathology possible, that we can use in another type of training
ddf = ddf.drop("INITIAL_EVIDENCE", axis=1)
ddf = ddf.drop("DIFFERENTIAL_DIAGNOSIS", axis=1)
# And of course the EVIDENCES column
ddf = ddf.drop("EVIDENCES", axis=1)

In [52]:
ddf.head()

,AGE,SEX,PATHOLOGY,z80.0,trav1_@_AmerN,protu_langue,lesions_peau_elevee_@_7,z77.22,lesions_peau_elevee_@_6,perinatality,...,lesions_peau_prurit_@_2,oedeme_endroitducorps_@_coté_lateral_du_pied_G_,douleurxx_endroitducorps_@_aisselle_D_,ains,douleurxx_endroitducorps_@_amygdale_D_,cont_coq,oedeme,boire_ped,douleurxx_endroitducorps_@_face_dorsale_du_poignet_D_,oedeme_endroitducorps_@_plante_du_pied_G_
0,18,1,IVRS ou virémie,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,21,1,VIH (Primo-infection),0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,19,0,Pneumonie,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,34,0,IVRS ou virémie,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,36,1,IVRS ou virémie,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
# Save the dataset into one csv for better control with pandas
ddf.to_csv('/content/gdrive/MyDrive/Colab/processed_dataset.csv', single_file=True, index=False)

['/content/gdrive/MyDrive/Colab/processed_dataset.csv']

The collaborative dataset is ready

## Model selection

In [4]:
# import necessary libraries
from sklearn.model_selection import train_test_split

from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import BernoulliNB
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import accuracy_score, classification_report

In [3]:
df = pd.read_csv('/content/gdrive/MyDrive/Colab/processed_dataset.csv', chunksize=129257)

In [4]:
classes = set()
for chunk in df:
  classes.update(chunk['PATHOLOGY'].unique())
classes = list(classes)

In [5]:
# Split the chunks to training and testing sets
df = pd.read_csv('/content/gdrive/MyDrive/Colab/processed_dataset.csv', chunksize=129257)
X_train_global = []
y_train_global = []
X_test_global = []
y_test_global = []
for chunk in df:
    X = chunk.drop('PATHOLOGY', axis=1)
    y = chunk['PATHOLOGY']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

    # save the training data
    X_train_global.append(X_train)
    y_train_global.append(y_train)

    # save the test data
    X_test_global.append(X_test)
    y_test_global.append(y_test)

# concat the test data as we will need it together for testing
X_test_global = pd.concat(X_test_global)
y_test_global = pd.concat(y_test_global)

In [12]:
def fit_by_chunks(model, features, targets):
  if(len(features) != len(targets)):
    raise Exception("features should be compatible with targets")

  for i in range(len(features)):
    print('Training model on chunk ', i+1)
    # Train the model by parts
    if i == 0: # pass the classes list in the 1st iteration
      model.partial_fit(features[i], targets[i], classes=classes)
    else:
      model.partial_fit(features[i], targets[i])
    print('Training on chunk ', i+1, ' finished')

    y_pred_test = model.predict(X_test_global)
    test_accuracy = accuracy_score(y_test_global, y_pred_test)
    print('Test Accuracy:', test_accuracy)
    print('-------------------------------------------------------')

### SGDClassifier

In [13]:
sgd = SGDClassifier()

In [14]:
fit_by_chunks(sgd, X_train_global, y_train_global)

Training model on chunk  1
Training on chunk  1  finished
Test Accuracy: 0.9155004216275597
-------------------------------------------------------
Training model on chunk  2
Training on chunk  2  finished
Test Accuracy: 0.9325589311547954
-------------------------------------------------------
Training model on chunk  3
Training on chunk  3  finished
Test Accuracy: 0.9282382157031123
-------------------------------------------------------
Training model on chunk  4
Training on chunk  4  finished
Test Accuracy: 0.9489327794152915
-------------------------------------------------------
Training model on chunk  5
Training on chunk  5  finished
Test Accuracy: 0.9671053140545098
-------------------------------------------------------
Training model on chunk  6
Training on chunk  6  finished
Test Accuracy: 0.9671285229110095
-------------------------------------------------------
Training model on chunk  7
Training on chunk  7  finished
Test Accuracy: 0.9489830652710408
--------------------

### Perceptron

In [15]:
pm = Perceptron()

In [16]:
fit_by_chunks(pm, X_train_global, y_train_global)

Training model on chunk  1
Training on chunk  1  finished
Test Accuracy: 0.9564988666341743
-------------------------------------------------------
Training model on chunk  2
Training on chunk  2  finished
Test Accuracy: 0.9635891722948143
-------------------------------------------------------
Training model on chunk  3
Training on chunk  3  finished
Test Accuracy: 0.9726947803281732
-------------------------------------------------------
Training model on chunk  4
Training on chunk  4  finished
Test Accuracy: 0.974582433990144
-------------------------------------------------------
Training model on chunk  5
Training on chunk  5  finished
Test Accuracy: 0.9786207750210814
-------------------------------------------------------
Training model on chunk  6
Training on chunk  6  finished
Test Accuracy: 0.9814174422292881
-------------------------------------------------------
Training model on chunk  7
Training on chunk  7  finished
Test Accuracy: 0.980450406541803
----------------------

### Naive Bayes

In [17]:
nb = BernoulliNB()

In [18]:
fit_by_chunks(nb, X_train_global, y_train_global)

Training model on chunk  1
Training on chunk  1  finished
Test Accuracy: 0.9973696629300408
-------------------------------------------------------
Training model on chunk  2
Training on chunk  2  finished
Test Accuracy: 0.9974160806430401
-------------------------------------------------------
Training model on chunk  3
Training on chunk  3  finished
Test Accuracy: 0.9974006080720403
-------------------------------------------------------
Training model on chunk  4
Training on chunk  4  finished
Test Accuracy: 0.9974160806430401
-------------------------------------------------------
Training model on chunk  5
Training on chunk  5  finished
Test Accuracy: 0.9974392894995396
-------------------------------------------------------
Training model on chunk  6
Training on chunk  6  finished
Test Accuracy: 0.9974392894995396
-------------------------------------------------------
Training model on chunk  7
Training on chunk  7  finished
Test Accuracy: 0.99742381692854
----------------------

### Multi-Layer Perceptron

In [19]:
mlp = MLPClassifier()

In [20]:
fit_by_chunks(mlp, X_train_global, y_train_global)

Training model on chunk  1
Training on chunk  1  finished
Test Accuracy: 0.9960893076798106
-------------------------------------------------------
Training model on chunk  2
Training on chunk  2  finished
Test Accuracy: 0.993358398898353
-------------------------------------------------------
Training model on chunk  3
Training on chunk  3  finished
Test Accuracy: 0.9973464540735412
-------------------------------------------------------
Training model on chunk  4
Training on chunk  4  finished
Test Accuracy: 0.9973619266445409
-------------------------------------------------------
Training model on chunk  5
Training on chunk  5  finished
Test Accuracy: 0.9973464540735412
-------------------------------------------------------
Training model on chunk  6
Training on chunk  6  finished
Test Accuracy: 0.9974624983560393
-------------------------------------------------------
Training model on chunk  7
Training on chunk  7  finished
Test Accuracy: 0.9954046464130712
---------------------

### Compare models

In [21]:
# list of models
models = [sgd, pm, nb, mlp]
results = []

# Evaluate each model on the test set
for model in models:
    y_pred = model.predict(X_test_global)
    accuracy = accuracy_score(y_test_global, y_pred)
    report = classification_report(y_test_global, y_pred, output_dict=True)
    
    results.append({
        'Model': type(model).__name__,
        'Accuracy': accuracy,
        'Precision': report['weighted avg']['precision'],
        'Recall': report['weighted avg']['recall'],
        'F1-Score': report['weighted avg']['f1-score']
    })

# Create a DataFrame from the results
df_results = pd.DataFrame(results)

In [22]:
df_results.head()

,Model,Accuracy,Precision,Recall,F1-Score
0,SGDClassifier,0.959369,0.989798,0.959369,0.969244
1,Perceptron,0.982106,0.988327,0.982106,0.981558
2,BernoulliNB,0.997428,0.997577,0.997428,0.997396
3,MLPClassifier,0.997451,0.997631,0.997451,0.997417


In [24]:
# save the models
import pickle
pickle.dump(sgd, open('/content/gdrive/MyDrive/Colab/models/sgd_model.pkl', 'wb'))
pickle.dump(pm, open('/content/gdrive/MyDrive/Colab/models/pm_model.pkl', 'wb'))
pickle.dump(nb, open('/content/gdrive/MyDrive/Colab/models/nb_model.pkl', 'wb'))
pickle.dump(mlp, open('/content/gdrive/MyDrive/Colab/models/mlp_model.pkl', 'wb'))

# Test the models (independent part)

In [8]:
import random
import pandas as pd
import numpy as np

In [16]:
import pickle
sgd = pickle.load(open('/content/gdrive/MyDrive/Colab/models/sgd_model.pkl', 'rb'))
pm = pickle.load(open('/content/gdrive/MyDrive/Colab/models/pm_model.pkl', 'rb'))
nb = pickle.load(open('/content/gdrive/MyDrive/Colab/models/nb_model.pkl', 'rb'))
mlp = pickle.load(open('/content/gdrive/MyDrive/Colab/models/mlp_model.pkl', 'rb'))

In [42]:
scores = pd.DataFrame(columns=['Model', 'Score'])
scores.loc[0] = [type(sgd).__name__, sgd.score(X_test_global, y_test_global)]
scores.loc[1] = [type(pm).__name__, pm.score(X_test_global, y_test_global)]
scores.loc[2] = [type(nb).__name__, nb.score(X_test_global, y_test_global)]
scores.loc[3] = [type(mlp).__name__, mlp.score(X_test_global, y_test_global)]
scores

,Model,Score
0,SGDClassifier,0.959369
1,Perceptron,0.982106
2,BernoulliNB,0.997428
3,MLPClassifier,0.997451


In [43]:
# save scores for later use
scores.to_csv('/content/gdrive/MyDrive/Colab/models/models_score.csv', index=False)

In [13]:
columns = []
with open('/content/gdrive/MyDrive/Colab/evidences.txt', 'r', encoding='utf-8') as f:
    # the evidences are stored as a list in the file, so we need to convert it to a list
    columns = ast.literal_eval(f.read())

In [39]:
# random number of symptoms but  not equal to the number of symptoms in the dataset
symptoms_num = 7
#random.randint(1, 30)
# random symptoms
symptoms = []
for i in range(symptoms_num):
    symptoms.append(columns[random.randint(0, len(columns)-1)])
# we need to create a dataframe with the same columns as the dataset
symptoms_df = pd.DataFrame(columns=X_test_global.columns)
# we need to add the symptoms to the dataframe
symptoms_df.loc[0] = 0
for symptom in symptoms:
    symptoms_df[symptom] = 1

# Other columns
symptoms_df['AGE'] = random.randint(5, 60)
symptoms_df['SEX'] = random.randint(0, 1)

symptoms

['oedeme_endroitducorps_@_mollet_D_',
 'i10',
 'lesions_peau_endroitducorps_@_palais',
 'douleurxx_carac_@_une_brûlure_ou_chaleur',
 'douleurxx_endroitducorps_@_omoplate_G_',
 'douleurxx_intens_@_2',
 'douleurxx_endroitducorps_@_tibia_G_']

In [40]:
# predict the disease using both models
disease = sgd.predict(symptoms_df)
# confidence = np.max(sgd.predict_proba(symptoms_df)) 

disease1 = pm.predict(symptoms_df)
# confidence1 = np.max(pm.predict_proba(symptoms_df))

disease2 = nb.predict(symptoms_df)
# confidence2 = np.max(nb.predict_proba(symptoms_df))

disease3 = mlp.predict(symptoms_df)
# confidence3 = np.max(mlp.predict_proba(symptoms_df))

In [41]:
# Let's plot a table with the results of the 5 models and the confidence of each model
results = pd.DataFrame(columns=['Model', 'Predicted Disease'])
results.loc[0] = [type(sgd).__name__, disease[0]]
results.loc[1] = [type(pm).__name__, disease1[0]]
results.loc[2] = [type(nb).__name__, disease2[0]]
results.loc[3] = [type(mlp).__name__, disease3[0]]
results

,Model,Predicted Disease
0,SGDClassifier,Laryngo-trachéo-bronchite (Croup)
1,Perceptron,Fibrillation auriculaire/Flutter auriculaire
2,BernoulliNB,Laryngo-trachéo-bronchite (Croup)
3,MLPClassifier,Fibrillation auriculaire/Flutter auriculaire
